In [1]:
import os
from collections import defaultdict

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'permutations.py',
 'Python_CS5001_Final_Exam_Q3_Lak.ipynb',
 'story.txt',
 'words.txt']

In [3]:
# Name: Lakhmipriya Anil

import string
from permutations import get_permutations

### HELPER CODE ###
def load_words(file_name):
    '''
    file_name (string): the name of the file containing
    the list of words to load

    Returns: a list of valid words. Words are strings of lowercase letters.

    Depending on the size of the word list, this function may
    take a while to finish.
    '''

    print("Loading word list from file...")
    # inFile: file
    inFile = open(file_name, 'r')
    # wordlist: list of strings
    wordlist = []
    for line in inFile:
        wordlist.extend([word.lower() for word in line.split(' ')])
    print("  ", len(wordlist), "words loaded.")
    return wordlist

def is_word(word_list, word):
    '''
    Determines if word is a valid word, ignoring
    capitalization and punctuation

    word_list (list): list of words in the dictionary.
    word (string): a possible word.

    Returns: True if word is in word_list, False otherwise

    Example:
    >>> is_word(word_list, 'bat') returns
    True
    >>> is_word(word_list, 'asdf') returns
    False
    '''
    word = word.lower()
    word = word.strip(" !@#$%^&*()-_+={}[]|\:;'<>?,./\"")
    return word in word_list


### END HELPER CODE ###

## Class SubMessage

In [47]:
WORDLIST_FILENAME = 'words.txt'

# you may find these constants helpful
VOWELS_LOWER = 'aeiou'
VOWELS_UPPER = 'AEIOU'
CONSONANTS_LOWER = 'bcdfghjklmnpqrstvwxyz'
CONSONANTS_UPPER = 'BCDFGHJKLMNPQRSTVWXYZ'

class SubMessage(object):
    def __init__(self, text):
        '''
        Initializes a SubMessage object

        text (string): the message's text

        A SubMessage object has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
        '''
        # Initializing instance variables
        self.message_text = text
        self.valid_words = load_words('words.txt')


    def get_message_text(self):
        '''
        Used to safely access self.message_text outside of the class

        Returns: self.message_text
        '''
        return self.message_text


    def get_valid_words(self):
        '''
        Used to safely access a copy of self.valid_words outside of the class.
        This helps you avoid accidentally mutating class attributes.

        Returns: a COPY of self.valid_words
        '''
        return self.valid_words.copy()


    def build_transpose_dict(self, vowels_permutation):
        '''
        vowels_permutation (string): a string containing a permutation of vowels (a, e, i, o, u)

        Creates a dictionary that can be used to apply a cipher to a letter.
        The dictionary maps every uppercase and lowercase letter to an
        uppercase and lowercase letter, respectively. Vowels are shuffled
        according to vowels_permutation. The first letter in vowels_permutation
        corresponds to a, the second to e, and so on in the order a, e, i, o, u.
        The consonants remain the same. The dictionary should have 52
        keys of all the uppercase letters and all the lowercase letters.

        Example: When input "eaiuo":
        Mapping is a->e, e->a, i->i, o->u, u->o
        and "Hello World!" maps to "Hallu Wurld!"

        Returns: a dictionary mapping a letter (string) to
        another letter (string).
        '''
        transpose_dict = defaultdict(str)
        
        n_vowels = len(VOWELS_LOWER)
        n_consonants = len(CONSONANTS_LOWER)

        for idx in range(n_vowels):
            vowel_ch_lower = VOWELS_LOWER[idx]
            vowel_ch_upper = VOWELS_UPPER[idx]
            encoded_vowel_ch = vowels_permutation[idx]
            transpose_dict[vowel_ch_lower] = encoded_vowel_ch
            transpose_dict[vowel_ch_upper] = encoded_vowel_ch.upper()
        
        for consonant in CONSONANTS_LOWER + CONSONANTS_UPPER:
            transpose_dict[consonant] = consonant
        
        return transpose_dict


    def apply_transpose(self, transpose_dict):
        '''
        transpose_dict (dict): a transpose dictionary

        Returns: an encrypted version of the message text, based
        on the dictionary
        '''
        message = self.message_text

        encoded_message = ''
        for char in message:
            if char in transpose_dict:
                encoded_message += transpose_dict.get(char)
            else:
                encoded_message += char

        return encoded_message


if __name__ == '__main__':
    # Test Case 1:
    # Input: 'Hello World!', vowels_permutation = 'eaiuo'
    # Expected Output: 'Hallu Wurld!'
    print('Test Case 1 - ')
    sub_message1 = SubMessage('Hello World!')
    transpose_dict1 = sub_message1.build_transpose_dict('eaiuo')
    encrypted_message1 = sub_message1.apply_transpose(transpose_dict1)
    # Actual Output: 'Hallu Wurld!'
    print('Encrypted Message:', encrypted_message1)
    
    print('\n-----------------------------------------------------------------------\n')

    # Test Case 2:
    # Input: 'Python is fun!', vowels_permutation = 'oiaue'
    # Expected Output: 'Pythun as fen!'
    print('Test Case 2 - ')
    sub_message2 = SubMessage('Python is fun!')
    transpose_dict2 = sub_message2.build_transpose_dict('oiaue')
    encrypted_message2 = sub_message2.apply_transpose(transpose_dict2)
    # Actual Output: 'Pythun as fen!'
    print('Encrypted Message:', encrypted_message2)
    
    
    

Test Case 1 - 
Loading word list from file...
   55901 words loaded.
Encrypted Message: Hallu Wurld!

-----------------------------------------------------------------------

Test Case 2 - 
Loading word list from file...
   55901 words loaded.
Encrypted Message: Pythun as fen!


## Class EncryptedSubMessage

In [48]:
class EncryptedSubMessage(SubMessage):
    def __init__(self, text):
        '''
        Initializes an EncryptedSubMessage object

        text (string): the encrypted message text

        An EncryptedSubMessage object inherits from SubMessage and has two attributes:
            self.message_text (string, determined by input text)
            self.valid_words (list, determined using helper function load_words)
        '''
        super().__init__(text)      # Initializing using constructor of parent class SubMessage

    def decrypt_message(self):
        '''
        Attempt to decrypt the encrypted message

        Idea is to go through each permutation of the vowels and test it
        on the encrypted message. For each permutation, check how many
        words in the decrypted text are valid English words, and return
        the decrypted message with the most English words.

        If no good permutations are found (i.e. no permutations result in
        at least 1 valid word), return the original string. If there are
        multiple permutations that yield the maximum number of words, return any
        one of them.

        Returns: the best decrypted message

        Hint: use the function from permutations
        '''

        encrypted_message = self.get_message_text()
        valid_word_cnt_dict = defaultdict(int)
        decrypted_message_dict = defaultdict(str)

        vowel_all_permutations = get_permutations(VOWELS_LOWER)
        for vowel_permutation in vowel_all_permutations:
            vowel_dict = self.build_transpose_dict(vowel_permutation)

            decrypted_message = self.apply_transpose(vowel_dict)
            decrypted_message_dict[vowel_permutation] = decrypted_message

            decrypted_words = decrypted_message.split(' ')
            valid_english_words = self.get_valid_words()

            for decrypt_word in decrypted_words:
                if is_word(valid_english_words, decrypt_word):
                    valid_word_cnt_dict[vowel_permutation] += 1

        if len(valid_word_cnt_dict) != 0:
            #print(valid_word_cnt_dict)
            #print('\ndecrypted_message_dict', decrypted_message_dict)
            max_valid_permute = max(valid_word_cnt_dict, key= lambda x: valid_word_cnt_dict[x])
            n_valid_words = valid_word_cnt_dict.get(max_valid_permute)
            max_valid_decrypt_msg = decrypted_message_dict[max_valid_permute]

            print(f'\nThe maximum number of valid English words is {n_valid_words} for vowel permutation \'{max_valid_permute}\'\n')
            return max_valid_decrypt_msg
                            # Returns the decrypted message with the most English words
        else:
            return encrypted_message


if __name__ == '__main__':
    # Test Case 1:
    # Input: 'Pythun os fio!' (encrypted message)
    # Expected Output: Python is fun! (decrypted message)
    print('Test Case 1 -')
    encrypted_message1 = 'Pythun os fin!'
    encrypted_sub_message1 = EncryptedSubMessage(encrypted_message1)
    decrypted_message1 = encrypted_sub_message1.decrypt_message()
    # Actual Output: 'Python us fiu!'
    print('Decrypted Message:', decrypted_message1)

    print('\n-----------------------------------------------------------------------\n')
    # Test Case 2:
    # Input: 'Hillo Wurld!' (encrypted message)
    # Expected Output: 'Hello World!' (decrypted message)
    print('Test Case 2 -')
    encrypted_message2 = 'Hillo Wurld!'   
    encrypted_sub_message2 = EncryptedSubMessage(encrypted_message2)
    decrypted_message2 = encrypted_sub_message2.decrypt_message()
    # Actual Output: 'Hello Wurld!'
    print('Decrypted Message:', decrypted_message2)
    
    print('\n-----------------------------------------------------------------------\n')
    # Test Case 3:
    # Input: story.txt (encrypted message)
    # Expected Output: (decrypted message)
    print('Test Case 3 -')
    print('Loading encrypted message from file story.txt...')
    encryptedFile = open('story.txt', 'r')
    encrypted_sub_message3 = EncryptedSubMessage(encryptedFile)
    decrypted_message3 = encrypted_sub_message3.decrypt_message()
    # Actual Output:
    print('Decrypted Message:', decrypted_message3)
          
    print('\nThe valid words are:')
    for w in decrypted_message3.split(' '):
        if w in encrypted_sub_message3.get_valid_words():
            print(w)


Test Case 1 -
Loading word list from file...
   55901 words loaded.

The maximum number of valid English words is 2 for vowel permutation 'aeiuo'

Decrypted Message: Python us fin!

-----------------------------------------------------------------------

Test Case 2 -
Loading word list from file...
   55901 words loaded.

The maximum number of valid English words is 1 for vowel permutation 'aieou'

Decrypted Message: Hello Wurld!

-----------------------------------------------------------------------

Test Case 3 -
Loading encrypted message from file story.txt...
Loading word list from file...
   55901 words loaded.

The maximum number of valid English words is 1 for vowel permutation 'aeiou'

Decrypted Message: Xoqy Tzcfsm wg o amhvwqoz qvofoqhsf qfsohsr cb hvs gdif ct o acasbh hc vszd qcjsf ob wbgittwqwsbhzm dzobbsr voqy. Vs vog pssb fsuwghsfsr tcf qzoggsg oh AWH hkwqs pstcfs, pih vog fsdcfhsrzm bsjsf doggsr oqzogg. Wh vog pssb hvs hforwhwcb ct hvs fsgwrsbhg ct Sogh Qoadig hc psqcas